# Imports

In [ ]:
import packages
from configs import settings, const, components
from configs.settings import logger
import asyncio, os, time, yaml, json, datetime, copy, random
from typing import Any, AsyncGenerator, Generator, Callable, Literal, Optional, TypeAlias, Union
from tqdm import tqdm
from pprint import pprint

from toolkit.llm.llama_index import (
	agents, cores, deploys as dpls, evaluation, messages, models, 
	observability, types, utils as utils_llama_index, workflows as wfs
)
from toolkit.llm.llama_index.data import loading, querying, storing

from features.agents.car.tools import VehicleDB
from features.agents.tools import map

from toolkit.utils import utils, typer as t
from toolkit.utils.llm import measure_performance, main as utils_llm
from toolkit.utils.utils import rp_print

# RAG

<span style="color: red;">We are currently in the migration process.<br>
The cell below contains the implementation of the LlamaIndex framework.<br>
You may not need to execute this. For more information on LangChain implementation, please refer to the URLs listed in the Reference section.
</span>

Please use LangChain version:
```
pip install --upgrade -r apps/requirements/dev.langchain.requirements.txt
```

In [ ]:
prompts_rag = settings.prompts_rag

prompt_generate_result = prompts_rag["generate_result"]

queries = [
	"Could you provide some tips or notes to consider before driving a car?",
	"How can parents prevent children from accidentally opening doors or windows while driving?",
	"When should a child start using the vehicle's seat belt instead of a child restraint system?",
	"What is the recommended seating position for children in a vehicle?",
]

user_query = queries[1]

retrieved_data = await utils.time_it(
	lambda: await components.retriever_car_manual.aretrieve(user_query), 
 	name="Vector Database Data Retrieval"
)

# retrieved_data = await components.retriever_car_manual.aretrieve(user_query)

retrieved_data_texts = await utils_llama_index.extract_text(retrieved_data)

async for token in await utils_llama_index.interact_model(
	prompt=prompt_generate_result, system_prompt=prompts_rag["system"]["car_manual"],
	mode="astream", 
	user_question=user_query, retrieved_data=retrieved_data_texts,
	measure_performance=True,
):
  print(token, end="", flush=True)

# Ref

- https://python.langchain.com/docs/tutorials/rag/
- https://python.langchain.com/docs/tutorials/qa_chat_history/
- https://python.langchain.com/docs/integrations/vectorstores/qdrant/
- Graph
  - https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/
  - https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/
  - https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/
  - https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/
  - https://langchain-ai.github.io/langgraph/tutorials/sql-agent/

Note:
- For a quick start, utilize the OpenAI GPT-3.5 model: https://python.langchain.com/docs/integrations/llms/openai/

Migrate from LlamaIndex to LangChain

- https://docs.llamaindex.ai/en/stable/examples/vector_stores/QdrantIndexDemo/
- https://docs.llamaindex.ai/en/stable/understanding/rag/
- https://docs.llamaindex.ai/en/stable/understanding/agent/rag_agent/
- https://docs.llamaindex.ai/en/stable/use_cases/q_and_a/
- module_guides
  - https://docs.llamaindex.ai/en/stable/module_guides/loading/
  - https://docs.llamaindex.ai/en/stable/module_guides/indexing/
  - https://docs.llamaindex.ai/en/stable/module_guides/storing/
  - https://docs.llamaindex.ai/en/stable/module_guides/querying/
- Examples
  - Workflows
    - https://docs.llamaindex.ai/en/stable/examples/workflow/citation_query_engine/
    - https://docs.llamaindex.ai/en/stable/examples/workflow/corrective_rag_pack/
    - https://docs.llamaindex.ai/en/stable/examples/workflow/long_rag_pack/
    - https://docs.llamaindex.ai/en/stable/examples/workflow/rag/